In [2]:
import os
import torch
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.utils.data as data
from PIL import Image
from tqdm import tqdm 
# from loss.SoftTriple import SoftTriple
from loss.softtriple import SoftTriple
import evaluation as eva
# import net
# from net.bninception import bninception
from net.cnn_ensemble import cnn_ensemble
from dataset.dataset_kmu_cnn import Dataset
# from net.resnet_model import ResNet50
# from net.resnet_model_cls import ResNet50
# from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt

In [2]:
datasettttt = '/root/notebooks/nfs/work/jason.chen/project/siamese-pytorch/dataset/KMU_dataset_512/BF_a_black1'


traindir = os.path.join(datasettttt, 'train')
testdir = os.path.join(datasettttt, 'val')

In [3]:
train_dataset   = Dataset(traindir, shuffle_pairs=True, augment=True)
val_dataset     = Dataset(testdir, shuffle_pairs=True, augment=False)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=24,num_workers=8, drop_last=True)
val_dataloader   = torch.utils.data.DataLoader(val_dataset, batch_size=6,num_workers=8, drop_last=True)

In [4]:
(image1,image2), y, (class1, class2) = next(iter(train_dataloader))

In [5]:
freeze_BN = True
gpu = 0

In [6]:
image1, image2, y = map(lambda x: x.to(gpu), [image1, image2, y])

In [7]:
label = []
IDimg_num = 3
for i in range (int(len(y)/IDimg_num)):
    yy = y[(IDimg_num*i)]
    
    if i >0:
        label = torch.cat((label,yy),0)
    else:
        label = yy
# label = label.unsqueeze(1)
label = label.to(gpu)

In [8]:
label = label.to(torch.long)

In [9]:
model = cnn_ensemble()
torch.cuda.set_device(0)
model = model.cuda()

In [10]:
criterion = SoftTriple().cuda()
optimizer = torch.optim.Adam([{"params": model.parameters(), "lr": 0.0001},
                              {"params": criterion.parameters(), "lr": 0.01}],
                             eps=0.01, weight_decay=1e-4)

In [11]:
output = model(image1, image2, batch_size=24)

In [12]:
output.shape

torch.Size([8, 512])

In [13]:
output.shape

torch.Size([8, 512])

In [14]:
label.shape

torch.Size([8])

In [12]:
loss, preds = criterion(output, label)

In [17]:
preds

tensor([0, 1, 1, 0, 1, 1, 0, 1], device='cuda:0')

In [19]:
label

tensor([0, 0, 1, 1, 0, 0, 1, 0], device='cuda:0')

In [16]:
loss

tensor(0.9110, device='cuda:0', grad_fn=<AddBackward0>)

In [14]:
def train(train_loader,correct,total,running_loss, model, criterion, optimizer,freeze_BN,gpu):
    # switch to train mode
    model.train()
    if freeze_BN:
        for m in model.modules():
            if isinstance(m, nn.BatchNorm2d):
                m.eval()
                
    for (img1, img2), y, (class1, class2) in train_loader:
        img1, img2, y = map(lambda x: x.to(device), [img1, img2, y])

        target = []
        IDimg_num = 3
        for i in range (int(len(y)/IDimg_num)):
            yy = y[(IDimg_num*i)]

            if i >0:
                target = torch.cat((target,yy),0)
            else:
                target = yy
        target = target.to(device)

        output = model(img1, img2, batch_size=24)
        loss, preds = criterion(output, label)
    
#         import pdb; pdb.set_trace()
        correct += (preds == target).sum().item()
        total += len(target)

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
    acc = correct / total
    train_epoch_loss = running_loss / len(train_loader)
    
    print('train_acc : ',acc ,'  train_loss : ',train_epoch_loss)
    return acc , train_epoch_loss

In [15]:
def validate(test_loader,correct,total,running_loss, model,criterion,gpu):
    # switch to evaluation mode
    model.eval()
    testdata = torch.Tensor()
    testlabel = torch.LongTensor()
    with torch.no_grad():
        for i, (input, target) in enumerate(test_loader):
            if gpu is not None:
                input = input.cuda(gpu, non_blocking=True)
                target = target.cuda(gpu, non_blocking=True)
            # compute output
            output = model(input)
            #         loss = criterion(output, target)
            loss, preds = criterion(output, target)
            correct += (preds == target).sum().item()
            total += len(target)
            running_loss += loss.item()
            
            testdata = torch.cat((testdata, output.cpu()), 0)
            testlabel = torch.cat((testlabel, target.cpu()))
            
        for (img1, img2), y, (class1, class2) in test_loader:
            img1, img2, y = map(lambda x: x.to(device), [img1, img2, y])

            target = []
            IDimg_num = 3
            for i in range (int(len(y)/IDimg_num)):
                yy = y[(IDimg_num*i)]

                if i >0:
                    target = torch.cat((target,yy),0)
                else:
                    target = yy
            target = target.to(device)

            output = model(img1, img2, batch_size = args.epochs)
            loss, preds = criterion(output, label)
            correct += (preds == target).sum().item()
            total += len(target)
            running_loss += loss.item()
            
            testdata = torch.cat((testdata, output.cpu()), 0)
            testlabel = torch.cat((testlabel, target.cpu()))
            
    val_epoch_loss = running_loss / len(test_loader)
    val_acc = correct / total
    print('val_acc : ',val_acc, 'val_loss : ',val_epoch_loss)
    nmi, recall = eva.evaluation(testdata.numpy(), testlabel.numpy(), [1, 2, 4, 8])
    return nmi, recall , val_acc, val_epoch_loss

In [16]:
def adjust_learning_rate(optimizer, epoch):
    # decayed lr by 10 every 20 epochs
    if (epoch+1)%20 == 0:
        for param_group in optimizer.param_groups:
            param_group['lr'] *= 0.1

In [ ]:
val_loss_list=[]
train_loss_list=[]
val_acc_list=[]
train_acc_list=[]
out_path = '/root/notebooks/nfs/work/jason.chen/project/triplet_loss/pytorch-triplet-loss/SoftTriple/out_model/KMU'
#--train
for epoch in range(0, 5):
    correct = 0
    total = 0
    running_loss = 0.0
    print('Training in Epoch[{}]'.format(epoch))
    adjust_learning_rate(optimizer, epoch)

    # train for one epoch
    train_acc, train_loss = train(train_loader, correct,total,running_loss,model, criterion, optimizer,freeze_BN,gpu)

    train_loss_list.append(train_loss)
    train_acc_list.append(train_acc)

# evaluate on validation set
    correct = 0
    total = 0
    running_loss = 0.0
    nmi, recall ,val_acc , val_loss= validate(test_loader,correct,total,running_loss, model,criterion,gpu)

    val_acc_list.append(val_acc)
    val_loss_list.append(val_loss)
#--save final model 
plt.plot(val_loss_list, label='val_loss')
plt.plot(train_loss_list, label='train_loss')
plt.legend()
plt.title('training loss and val loss')
plt.savefig(os.path.join(out_path+'KMU', 'loss.png'))
plt.cla()
plt.plot(val_acc_list, label='val_acc')
plt.plot(train_acc_list, label='train_acc')
plt.legend()
plt.title('training acc and val acc')
plt.savefig(os.path.join(out_path+'KMU', 'acc.png'))

torch.save(
            {
                "epoch": epoch + 1,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict()
            },
            os.path.join(out_path+'KMU', 'KMU'+'.pth')
        )   
print('Recall@1, 2, 4, 8: {recall[0]:.3f}, {recall[1]:.3f}, {recall[2]:.3f}, {recall[3]:.3f}; NMI: {nmi:.3f} \n'
                  .format(recall=recall, nmi=nmi))

In [3]:
x = torch.randn(2, 3)
torch.cat((x, x), 1)

tensor([[ 0.8682,  0.2698, -0.1919,  0.8682,  0.2698, -0.1919],
        [-1.2108,  0.0146, -0.1874, -1.2108,  0.0146, -0.1874]])

In [4]:
max(1, len(x))

2